# Preparing for fine-tunning

## Tokenizing text

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from datasets import load_dataset 

train_data = load_dataset("imdb", split="train") 
train_data = train_data.shard(num_shards=4, index=0) 
test_data = load_dataset("imdb", split="test") 
test_data = test_data.shard(num_shards=4, index=0)

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased") 
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the data
tokenized_training_data = tokenizer(str(train_data["text"]), return_tensors="pt", padding=True, truncation=True, max_length=64)
tokenized_test_data = tokenizer(str(test_data["text"]), return_tensors="pt", padding=True, truncation=True, max_length=64)

print(tokenized_training_data)
print(tokenized_test_data)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[  101,  5930,  1006,  1031,  1005,  1045, 12524,  1045,  2572,  8025,
          1011,  3756,  2013,  2026,  2678,  3573,  2138,  1997,  2035,  1996,
          6704,  2008,  5129,  2009,  2043,  2009,  2001,  2034,  2207,  1999,
          3476,  1012,  1045,  2036,  2657,  2008,  2012,  2034,  2009,  2001,
          8243,  2011,  1057,  1012,  1055,  1012,  8205,  2065,  2009,  2412,
          2699,  2000,  4607,  2023,  2406,  1010,  3568,  2108,  1037,  5470,
          1997,  3152,  2641,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Mapping tokenization

In [2]:
# Complete the function
def tokenize_function(data):
    return tokenizer(data["text"], 
                     return_tensors="pt", 
                     padding=True, 
                     truncation=True, 
                     max_length=64)

tokenized_in_batches = train_data.map(tokenize_function, batched=True)

print(tokenized_in_batches)

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6250
})


In [3]:
# Complete the function
def tokenize_function(data):
    return tokenizer(data["text"], 
                     return_tensors="pt", 
                     padding=True, 
                     truncation=True, 
                     max_length=64)

# Tokenize row by row
tokenized_by_row = train_data.map(tokenize_function, batched=False)

print(tokenized_by_row)

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6250
})


# Fine-tunning through training

## Setting up training arguments

In [4]:
from transformers import Trainer,  TrainingArguments

# Set up an instance of TrainingArguments
training_args = TrainingArguments(   
  output_dir="./finetuned",
  eval_strategy="epoch",
  num_train_epochs=3,   
  learning_rate=2e-5,     
  per_device_train_batch_size=8,   
  per_device_eval_batch_size=8,
  weight_decay=0.01, 
)

## Setting up the trainer

In [5]:
# Set up the trainer object
trainer = Trainer(
    model=model,
    # Assign the training arguments and tokenizer
    args=training_args,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_test_data,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

/tmp/ipykernel_20123/3650103024.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/jorge/.pyenv/versions/hugging-face-fundamentals/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


IndexError: list index out of range

## Using the fine-tuned model

In [ ]:
import torch

input_text = ["I'd just like to say, I love the product! Thank you!"]

# Tokenize the new data
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Pass the tokenized inputs through the model
with torch.no_grad():
    outputs = model(**inputs)

# Extract the new predictions
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

label_map = {0: "Low risk", 1: "High risk"}
for i, predicted_label in enumerate(predicted_labels):
    churn_label = label_map[predicted_label]
    print(f"\n Input Text {i + 1}: {input_text[i]}")
    print(f"Predicted Label: {predicted_label}")

# Fine-tuning approaches

## Transfer learning with one-shot learning

In [ ]:
# Include an example in the input ext
input_text = """
Text: "The dinner we had was great and the service too."
Classify the sentiment of this sentence as either positive or negative.
Example:
Text: "The food was delicious"
Sentiment: Positive
Text: "The dinner we had was great and the service too."
Sentiment:
"""

# Apply the example to the model
result = model(input_text, max_length=100)

print(result[0]["label"])